In [8]:
from LightningModules import AutoEncoder, FUCCIDataModule, ReconstructionVisualization
from kornia import tensor_to_image
from microfilm.colorify import multichannel_to_rgb
import numpy as np
from PIL import Image
from pathlib import Path
import torch

In [9]:
FUCCI_PATH = '/home/ishang/cross-modal-autoencoders/FUCCI-dev-data'
LOGS = '/data/ishang/fucci_vae'
!ls {LOGS}

FUCCI_reference_VAE_2023_16_05_06_31  FUCCI_reference_VAE_2023_16_05_08_54
FUCCI_reference_VAE_2023_16_05_06_32  FUCCI_reference_VAE_2023_16_05_08_55
FUCCI_reference_VAE_2023_16_05_06_38  FUCCI_reference_VAE_2023_16_05_09_01
FUCCI_reference_VAE_2023_16_05_06_39  FUCCI_reference_VAE_2023_16_05_09_02
FUCCI_reference_VAE_2023_16_05_06_41  FUCCI_reference_VAE_2023_16_05_10_26
FUCCI_reference_VAE_2023_16_05_06_42  FUCCI_reference_VAE_2023_16_05_10_27
FUCCI_reference_VAE_2023_16_05_07_19  lightning_logs
FUCCI_reference_VAE_2023_16_05_07_22  logs
FUCCI_reference_VAE_2023_16_05_08_39  wandb_logs


In [10]:
model_folder = 'FUCCI_reference_VAE_2023_16_05_10_26'
!ls {LOGS}/{model_folder}/lightning_logs/

'epoch=88-Val_loss=0.00.ckpt'


In [15]:
checkpoint = 'epoch=88-Val_loss=0.00.ckpt'
model_path = Path(LOGS) / model_folder / "lightning_logs" / checkpoint
res = 64

In [12]:
# ckpt = torch.load(model_path)

In [13]:
# len(ckpt['state_dict'].keys())
# print(ckpt['state_dict'].keys())
# ckpt['state_dict']['decoder.layers.0.0.weight'].shape
# for k, v in ckpt['state_dict'].items():
#     print(k, v.shape)

In [16]:
model = AutoEncoder.load_from_checkpoint(model_path)

In [17]:
dm = FUCCIDataModule(
    data_dir=FUCCI_PATH,
    dataset="reference",
    imsize=res,
    split=(0.8, 0.1, 0.1),
    batch_size=8,
    num_workers=16
)

In [18]:
# print(dm.dataset[:3].shape)
x = dm.dataset[:3]
x_hat = model(x)
print(x.shape)
print(x_hat.shape)

torch.Size([3, 2, 64, 64])
torch.Size([3, 2, 64, 64])


In [22]:
grid = ReconstructionVisualization.make_reconstruction_grid(x, x_hat)

shape = grid.shape
for i in range(3):
    for j in range(2):
        y_st, y_end = i * shape[1] // 3, (i + 1) * shape[1] // 3
        x_st, x_end = j * shape[2] // 2, (j + 1) * shape[2] // 2
        for c in range(2):
            print(torch.min(grid[c, y_st:y_end, x_st:x_end]), torch.max(grid[c, y_st:y_end, x_st:x_end]))
    print()

print(grid.shape)
# img = np.moveaxis(tensor_to_image(grid), -1, 0)
img = grid.cpu().detach().numpy()
print(img.shape)
for i in range(2):
    image_composite, _, _, _= multichannel_to_rgb(img[i], cmaps=[dm.dataset.channel_colors()[i]])
    image_composite = (255 * image_composite).astype(np.uint8)
    image_composite = Image.fromarray(image_composite)
    image_composite.save(f'pred_grid_{res}_{i}_composite.png')
image_composite, _, _, _= multichannel_to_rgb(img, cmaps=dm.dataset.channel_colors())
image_composite = (255 * image_composite).astype(np.uint8)
image_composite = Image.fromarray(image_composite)
image_composite.save(f'pred_grid_{res}_composite.png')

data_grid = ReconstructionVisualization.make_reconstruction_grid(x, x)
print(data_grid.shape)
data_img = np.moveaxis(tensor_to_image(data_grid), -1, 0)
print(data_img.shape)
data_composite, _, _, _= multichannel_to_rgb(data_img, cmaps=dm.dataset.channel_colors())
data_composite = (255 * data_composite).astype(np.uint8)
data_composite = Image.fromarray(data_composite)
data_composite.save(f'data_grid_{res}_composite.png')

tensor(0.) tensor(0.5632)
tensor(0.) tensor(1.)
tensor(0.) tensor(0.4199)
tensor(0.) tensor(0.4124)

tensor(0.) tensor(0.5236)
tensor(0.) tensor(0.6551)
tensor(0.) tensor(0.4588)
tensor(0.) tensor(0.4510)

tensor(0.) tensor(0.8883)
tensor(0.) tensor(0.8469)
tensor(0.) tensor(0.6327)
tensor(0.) tensor(0.6214)

torch.Size([2, 200, 134])
(2, 200, 134)
torch.Size([2, 200, 134])
(2, 200, 134)
